In [ ]:
# Collection of algorithms on graphs.
# init

from .tarjan import *
from .check_bipartite import *
from .maximum_flow import *
from .maximum_flow_bfs import *
from .maximum_flow_dfs import *
from .all_pairs_shortest_path import *
from .bellman_ford import *
from .prims_minimum_spanning import *

## Undirected Graph using the Bron-Kerbosch Algorithm

In [ ]:
"""
Finds all cliques in an undirected graph. A clique is a set of vertices in the
graph such that the subgraph is fully connected (ie. for any pair of nodes in
the subgraph there is an edge between them).
"""

In [ ]:
def find_all_cliques(edges):
    """
    takes dict of sets
    each key is a vertex
    value is set of all edges connected to vertex
    returns list of lists (each sub list is a maximal clique)
    implementation of the basic algorithm described in:
    Bron, Coen; Kerbosch, Joep (1973), "Algorithm 457: finding all cliques of an undirected graph",
    """

    def expand_clique(candidates, nays):
        nonlocal compsub
        if not candidates and not nays:
            nonlocal solutions
            solutions.append(compsub.copy())
        else:
            for selected in candidates.copy():
                candidates.remove(selected)
                candidates_temp = get_connected(selected, candidates)
                nays_temp = get_connected(selected, nays)
                compsub.append(selected)
                expand_clique(candidates_temp, nays_temp)
                nays.add(compsub.pop())

    def get_connected(vertex, old_set):
        new_set = set()
        for neighbor in edges[str(vertex)]:
            if neighbor in old_set:
                new_set.add(neighbor)
        return new_set

    compsub = []
    solutions = []
    possibles = set(edges.keys())
    expand_clique(possibles, set())
    return solutions

## Graph Pathfinding Functions for Finding, Exploring, and Shortest Path Discovery

#### Find a path between two nodes using recursion and backtracking.

In [ ]:
# pylint: disable=dangerous-default-value
def find_path(graph, start, end, path=[]):
    
    path = path + [start]
    if start == end:
        return path
    if not start in graph:
        return None
    for node in graph[start]:
        if node not in path:
            newpath = find_path(graph, node, end, path)
            return newpath
    return None

#### Find all paths between two nodes using recursion and backtracking

In [ ]:
# pylint: disable=dangerous-default-value
def find_all_path(graph, start, end, path=[]):

    path = path + [start]
    if start == end:
        return [path]
    if not start in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path:
            newpaths = find_all_path(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

#### Find the shortest path between two nodes

In [ ]:
def find_shortest_path(graph, start, end, path=[]):
    
    path = path + [start]
    if start == end:
        return path
    if start not in graph:
        return None
    shortest = None
    for node in graph[start]:
        if node not in path:
            newpath = find_shortest_path(graph, node, end, path)
            if newpath:
                if not shortest or len(newpath) < len(shortest):
                    shortest = newpath
    return shortest


## Graph Representation Classes: Nodes, Edges, and Directed Graphs

#### A node/vertex in a graph.

In [ ]:
class Node:
  
    def __init__(self, name):
        self.name = name

    @staticmethod
    def get_name(obj):
        """
        Return the name of the node
        """
        if isinstance(obj, Node):
            return obj.name
        if isinstance(obj, str):
            return obj
        return''

    def __eq__(self, obj):
        return self.name == self.get_name(obj)

    def __repr__(self):
        return self.name

    def __hash__(self):
        return hash(self.name)

    def __ne__(self, obj):
        return self.name != self.get_name(obj)

    def __lt__(self, obj):
        return self.name < self.get_name(obj)

    def __le__(self, obj):
        return self.name <= self.get_name(obj)

    def __gt__(self, obj):
        return self.name > self.get_name(obj)

    def __ge__(self, obj):
        return self.name >= self.get_name(obj)

    def __bool__(self):
        return self.name

####  A directed edge in a directed graph. Stores the source and target node of the edge.

In [ ]:
class DirectedEdge:
    
    def __init__(self, node_from, node_to):
        self.source = node_from
        self.target = node_to

    def __eq__(self, obj):
        if isinstance(obj, DirectedEdge):
            return obj.source == self.source and obj.target == self.target
        return False

    def __repr__(self):
        return f"({self.source} -> {self.target})"

#### A directed graph. Stores a set of nodes, edges and adjacency matrix.

In [ ]:
class DirectedGraph:
   
    # pylint: disable=dangerous-default-value
    def __init__(self, load_dict={}):
        self.nodes = []
        self.edges = []
        self.adjacency_list = {}

        if load_dict and isinstance(load_dict, dict):
            for vertex in load_dict:
                node_from = self.add_node(vertex)
                self.adjacency_list[node_from] = []
                for neighbor in load_dict[vertex]:
                    node_to = self.add_node(neighbor)
                    self.adjacency_list[node_from].append(node_to)
                    self.add_edge(vertex, neighbor)

    def add_node(self, node_name):
        """
        Add a new named node to the graph.
        """
        try:
            return self.nodes[self.nodes.index(node_name)]
        except ValueError:
            node = Node(node_name)
            self.nodes.append(node)
            return node

    def add_edge(self, node_name_from, node_name_to):
        """
        Add a new edge to the graph between two nodes.
        """
        try:
            node_from = self.nodes[self.nodes.index(node_name_from)]
            node_to = self.nodes[self.nodes.index(node_name_to)]
            self.edges.append(DirectedEdge(node_from, node_to))
        except ValueError:
            pass

## Markov Chain

In [ ]:
"""
Chains are described using a dictionary:

    my_chain = {
        'A': {'A': 0.6,
              'E': 0.4},
        'E': {'A': 0.7,
              'E': 0.3}
    }
"""

In [ ]:
import random

#### Choose the next state randomly

In [ ]:
def __choose_state(state_map):
    
    choice = random.random()
    probability_reached = 0
    for state, probability in state_map.items():
        probability_reached += probability
        if probability_reached > choice:
            return state
    return None

#### Given a markov-chain, randomly chooses the next state given the current state.

In [ ]:
def next_state(chain, current_state):
   
    next_state_map = chain.get(current_state)
    return __choose_state(next_state_map)

#### Yield a sequence of states given a markov chain and the initial state

In [ ]:
def iterating_markov_chain(chain, state):
   
    while True:
        state = next_state(chain, state)
        yield state
